In [19]:
from moviepy.editor import AudioFileClip, CompositeAudioClip, concatenate_audioclips
from glob import glob
from math import floor
import os

org_files = list(glob('/workspace/demo/org/*'))
noise_files = list(glob('/workspace/demo/noise/*'))
compose_dir = '/workspace/demo/noised_comp'

print(org_files)
print(noise_files)

org_clips = [AudioFileClip(f) for f in org_files]
noise_clips = [AudioFileClip(f) for f in noise_files]

max_org_dur = max([c.duration for c in org_clips])
os.makedirs(compose_dir, exist_ok=True)
for i in range(len(noise_clips)):
    ns_clip = noise_clips[i]
    loop = floor(max_org_dur / ns_clip.duration)
    noise_clips[i] = concatenate_audioclips([ns_clip for _ in range(loop)])




['/workspace/demo/org/Scarborough.wav', '/workspace/demo/org/Sky Of More.mp3', '/workspace/demo/org/Darling, Without You.mp3', "/workspace/demo/org/He Loves I Don't Care.mp3", '/workspace/demo/org/晚风心里吹.mp3', '/workspace/demo/org/Apology Of Dreams.mp3', '/workspace/demo/org/Home Rhythm.mp3', "/workspace/demo/org/Babe, We're Crazy.mp3", '/workspace/demo/org/Best Moves.mp3', '/workspace/demo/org/Open Up To Her Tomorrow.mp3', '/workspace/demo/org/千里之外.mp3', '/workspace/demo/org/Dream Of My Home.mp3', '/workspace/demo/org/Season Of My Heart.mp3']
['/workspace/demo/noise/增强版黑胶唱片底噪.aiff', '/workspace/demo/noise/尘封的磁带录音.wav', '/workspace/demo/noise/黑胶唱片播放底噪.wav', '/workspace/demo/noise/空白黑胶唱片播放底噪.wav', '/workspace/demo/noise/黑胶唱片噪音.flac']


In [20]:
for org_fp, org_clip in zip(org_files, org_clips):
    for ns_fp, ns_clip in zip(noise_files, noise_clips):
        org_name = org_fp.split('/')[-1].split('.')[0]
        ns_name = ns_fp.split('/')[-1].split('.')[0]
        comp_fp = os.path.join(compose_dir, f"{org_name}-{ns_name}.mp3")
        comp_clip = CompositeAudioClip([org_clip.set_start(1.5), ns_clip.subclip(0, org_clip.duration)])
        comp_clip.write_audiofile(comp_fp, org_clip.fps, logger=None)
            



In [4]:
from glob import glob
import subprocess
compose_dir = '/workspace/demo/noised_comp'

def low_bit(fp, bitrate):
    lb_fp = fp[:-4] + f'-br_{bitrate}.mp3'
    cmd = f"ffmpeg -y -i {fp} -b:a {bitrate}k {lb_fp}"
    subprocess.run(cmd, shell=True)

for fp in glob(compose_dir+'/*.mp3'):
    if '-br_' in fp:
        continue
    low_bit(fp, 32)
    low_bit(fp, 64)
    low_bit(fp, 128)

    # lb_fp = fp[:-4] + '-br_64.mp3'
    # cmd = f"ffmpeg -y -i {fp} -b:a 64k {lb_fp}"
    # subprocess.run(cmd, shell=True)
    break


ffmpeg -y -i /workspace/demo/noised_comp/晚风心里吹-增强版黑胶唱片底噪.mp3 -b:a 64k /workspace/demo/noised_comp/晚风心里吹-增强版黑胶唱片底噪-br_64.mp3


ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
Input #0, mp3, from '/workspace/demo/noised_comp/晚风心里吹-增强版黑胶唱片底噪.mp3':
  Metadata:
    encoder         : Lavf58.29.100
  Duration: 00:03:07.98, start: 0.025057, bitrate: 